In [1]:

import numpy as np # linear algebra
import pandas as pd 


import os
for dirname, _, filenames in os.walk('netflix1.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:

from ydata_profiling import ProfileReport

from plotly.subplots import make_subplots
import plotly.graph_objects as go


from plotly.offline import plot, iplot, init_notebook_mode
import plotly.express as px
init_notebook_mode(connected=True)


c:\ProgramData\anaconda3\envs\profiling_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#data information
#reading the input file
df = pd.read_csv('netflix1.csv')
df.head(10)

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies"
5,s9,TV Show,The Great British Baking Show,Andy Devonshire,United Kingdom,9/24/2021,2021,TV-14,9 Seasons,"British TV Shows, Reality TV"
6,s10,Movie,The Starling,Theodore Melfi,United States,9/24/2021,2021,PG-13,104 min,"Comedies, Dramas"
7,s939,Movie,Motu Patlu in the Game of Zones,Suhas Kadav,India,5/1/2021,2019,TV-Y7,87 min,"Children & Family Movies, Comedies, Music & Mu..."
8,s13,Movie,Je Suis Karl,Christian Schwochow,Germany,9/23/2021,2021,TV-MA,127 min,"Dramas, International Movies"
9,s940,Movie,Motu Patlu in Wonderland,Suhas Kadav,India,5/1/2021,2013,TV-Y7,76 min,"Children & Family Movies, Music & Musicals"


In [ ]:
# To see the high level data details
#approach 1
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8790 non-null   object
 1   type          8790 non-null   object
 2   title         8790 non-null   object
 3   director      8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 686.8+ KB


In [ ]:
#approach 2
def missing_pct(df):
    # Calculate missing value and their percentage for each column
    missing_count_percent = df.isnull().sum() * 100 / df.shape[0]
    df_missing_count_percent = pd.DataFrame(missing_count_percent).round(2)
    df_missing_count_percent = df_missing_count_percent.reset_index().rename(
                    columns={
                            'index':'Column',
                            0:'Missing_Percentage (%)'
                    }
                )
    df_missing_value = df.isnull().sum()
    df_missing_value = df_missing_value.reset_index().rename(
                    columns={
                            'index':'Column',
                            0:'Missing_value_count'
                    }
                )
    # Sort the data frame
    Final = df_missing_value.merge(df_missing_count_percent, how = 'inner', left_on = 'Column', right_on = 'Column')
    Final = Final.sort_values(by = 'Missing_Percentage (%)',ascending = False)
    return Final

missing_pct(df)

,Column,Missing_value_count,Missing_Percentage (%)
0,show_id,0,0.0
1,type,0,0.0
2,title,0,0.0
3,director,0,0.0
4,country,0,0.0
5,date_added,0,0.0
6,release_year,0,0.0
7,rating,0,0.0
8,duration,0,0.0
9,listed_in,0,0.0


In [ ]:
#handling the missing data and deleting duplicates

df['rating'] = df['rating'].replace({'74 min': 'TV-MA', '84 min': 'TV-MA', '66 min': 'TV-MA'})
df['rating'] = df['rating'].replace({'TV-Y7-FV': 'TV-Y7'})

In [10]:
df['rating'].unique()

array(['PG-13', 'TV-MA', 'TV-PG', 'TV-14', 'TV-Y7', 'TV-Y', 'PG', 'TV-G',
       'R', 'G', 'NC-17', 'NR', 'UR'], dtype=object)

In [ ]:
# Renaming vaules for Rating for better understanding
df['rating'] = df['rating'].replace({
                'PG-13': 'Teens - Age above 12',
                'TV-MA': 'Adults',
                'PG': 'Kids - with parental guidence',
                'TV-14': 'Teens - Age above 14',
                'TV-PG': 'Kids - with parental guidence',
                'TV-Y': 'Kids',
                'TV-Y7': 'Kids - Age above 7',
                'R': 'Adults',
                'TV-G': 'Kids',
                 'G': 'Kids',
                'NC-17': 'Adults',
                'NR': 'NR',
                'UR' : 'UR'
                
})

In [12]:
df['rating'].unique()

array(['Teens - Age above 12', 'Adults', 'Kids - with parental guidence',
       'Teens - Age above 14', 'Kids - Age above 7', 'Kids', 'NR', 'UR'],
      dtype=object)

In [13]:
print(df.columns)

Index(['show_id', 'type', 'title', 'director', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in'],
      dtype='object')


In [14]:
# 1. Handle 'country' using mode
df['country'] = df['country'].fillna(df['country'].mode()[0])

# 2. Handle 'director' (This column EXISTS in your list)
df['director'] = df['director'].replace(np.nan, 'No Data')

# 3. Drop remaining NaNs and duplicates
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

print("Data cleaned successfully!")

Data cleaned successfully!


In [ ]:
# splitting the genres in different rows to use it in the viz later

df_genre = df[['show_id', 'title','type', 'listed_in' ]]
df_genre = (df_genre.drop('listed_in', axis=1)
             .join
             (
             df_genre.listed_in
             .str
             .split(', ',expand=True)
             .stack()
             .reset_index(drop=True, level=1)
             .rename('listed_in')           
             ))


In [16]:
# Creating new columns

df['month'] = pd.DatetimeIndex(df['date_added']).month

In [17]:
# Total Shows and movies

df_count = df['show_id'].count().sum()
print(df_count)
# Split of showes and TV
df_type = df.groupby('type')['show_id'].count().reset_index()
df_type = df_type.rename(columns = {"show_id":"count_showids"})

8790


In [18]:
%pip install nbformat --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
import sys
!{sys.executable} -m pip install nbformat --upgrade


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#Visualization
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook_connected'
fig = go.Figure()
fig.add_trace(go.Indicator(
    value = df_count))

fig = fig.update_layout(
        template = {'data' : {'indicator': [{
        'title': {'text': "Total content on Netflix"},}]
        }})

fig = fig.update_layout(
    height=100,
    margin=dict(l=50,r=50,b=0,t=1),)


fig.show()


fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]])
fig.add_trace(
    
    go.Bar(x= df_type['count_showids'], y= df_type['type'], orientation = 'h', marker=dict(color=["Maroon", "Grey"]), showlegend=False, 
           text = df_type['count_showids'], textposition='auto'),
    row=1, col=1)

fig.add_trace(
    
    go.Pie(labels=df_type['type'], values=df_type['count_showids'], marker_colors= ["Maroon", "Grey"]),
    row=1, col=2)

fig.update_layout(
    title_text="'What type of content is more uploaded more on Netflix?")
fig.show()


In [ ]:
# splitting the countries in different rows 
df_country = df[['show_id', 'title','type', 'country' ]]
df_country = (df_country.drop('country', axis=1)
             .join
             (
             df_country.country
             .str
             .split(', ',expand=True)
             .stack()
             .reset_index(drop=True, level=1)
             .rename('country')           
             ))


In [22]:
df_country_viz_total = df_country[["title", "country"]]
df_country_viz_total = df_country_viz_total.groupby(['country'])["title"].count().reset_index().sort_values('title', ascending= False).head(10)
df_country_viz_total = df_country_viz_total.rename(columns = {"title": "movies_count",})

 
fig1 = px.bar(df_country_viz_total, x='country', y='movies_count', color_discrete_sequence=px.colors.sequential.RdBu,
       title='Top 10 countries with Netflix Content ')


df_country_viz = df_country[["title", "country"]]
df_country_viz = df_country_viz.groupby(['country'])["title"].count().reset_index().sort_values('title', ascending= False).head(10)

df_country_viz1 = df_country[["title", "type", "country"]]
df_country_viz1 = df_country_viz1.groupby(['country', 'type'])["title"].count().reset_index().sort_values('title', ascending= False)
df_country_viz1 = df_country_viz1.rename(columns = {"title": "movies_count",})

final1 = df_country_viz.merge(df_country_viz1, how = 'left', left_on = 'country', right_on = 'country')
final1['percentage'] = (final1['movies_count']/final1['title'])*100
final1['percentage'] = final1['percentage'].round(1)
final1['percent_string'] = final1['percentage'].astype(str)+ '%'


fig2 = px.bar(final1, x='country', y='percentage', color = 'type',
       title='Top 10 countries with Movie/TV show split ')

In [23]:
fig = go.Figure()
fig.add_trace(
    
go.Bar(x= df_country_viz_total['country'], y= df_country_viz_total['movies_count'], marker_color = 'Maroon',
           text = df_country_viz_total['movies_count'], textposition='auto'))

fig.update_layout(title_text = "Top 10 countries with Netflix Content"
                  , yaxis=dict(title='Movies/TV Shows Count'))
fig.show()

final_movie = final1.query("type == 'Movie'")
final_show = final1.query("type == 'TV Show'")

fig = go.Figure()
fig.add_trace(go.Bar(
    x=  final_movie['country'],
    y= final_movie['percentage'],
    showlegend=True,
    text = final_movie['percent_string'], 
    textposition='auto',
    name='Movie',
    marker_color='Maroon'    
    
))
fig.add_trace(go.Bar(
    x= final_show['country'],
    y= final_show['percentage'],
    showlegend=True,
    text = final_show['percent_string'], 
    textposition='auto',
    name='TV Show',
    marker_color='Grey' 
))



# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='stack', title_text = 'Top 10 countries with Movie/TV show split '
                  , yaxis=dict(title='% Movies/TV Shows Count'))
fig.show()




In [24]:
df_country_viz = df_country[["title", "type", "country"]]
df_country_viz = df_country_viz.groupby(['country', 'type'])["title"].count().reset_index().sort_values('title', ascending= False)
df_country_viz = df_country_viz.rename(columns = {"title": "movies_count",})

df_country_movie = df_country_viz.query("type == 'Movie'").head(10) 
fig1 = px.bar(df_country_movie, x='country', y='movies_count',color_discrete_sequence=['Maroon'],
        title='Top 10 countries with the most Netflix movies')

df_country_movie = df_country_viz.query("type == 'TV Show'").head(10)
fig2 = px.bar(df_country_movie, x='country', y='movies_count', color_discrete_sequence=['gray'],
        title='Top 10 countries with the most Netflix TV Shows')

fig1.show()
fig2.show()

In [25]:
df_2 = df.query("type == 'Movie'")
df_2 = df_2[["title", "rating"]]
df_2 = df_2.groupby(['rating'])["title"].count().reset_index().sort_values('title', ascending = False)
df_2 = df_2.rename(columns = {"title": "movies_count"})
px.bar(df_2, x='rating', y='movies_count', color_discrete_sequence=px.colors.sequential.RdBu,
       title='For which category the maximum content(Movies) are uploaded? ')


In [26]:
df_3 = df.query("type == 'TV Show'")
df_3 = df_3[["title", "rating"]]
df_3 = df_3.groupby('rating')["title"].count().reset_index().sort_values('title', ascending = False)
df_3 = df_3.rename(columns = {"title": "movies_count"})
px.bar(df_3, x='rating', y='movies_count', color_discrete_sequence=['grey'],
       title='For which category the maximum content(TV Shows) are uploaded?')


In [27]:
df_5 = df.query("release_year >= 2007")
df_5 = df_5.groupby("release_year")["show_id"].count().reset_index()

fig = px.area(df_5, x='release_year', y='show_id', color_discrete_sequence=px.colors.sequential.RdBu,
      title='Overall content release Trend')
fig.show()

In [ ]:

df_4 = df.query("release_year >= 2007")
df_4 = df_4.groupby(["type","release_year"])["show_id"].count().reset_index()
df_4_movie = df_4.query("type == 'Movie'")
df_4_show = df_4.query("type == 'TV Show'")

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=  df_4_movie['release_year'],
    y= df_4_movie['show_id'],
    showlegend=True,
    text = df_4_movie['show_id'], 
    
    name='Movie',
    marker_color='Maroon'    
    
))
fig.add_trace(go.Scatter(
    x=  df_4_show['release_year'],
    y= df_4_show['show_id'],
    showlegend=True,
    text = df_4_show['show_id'], 
 
    name='TV Show',
    marker_color='Grey' 
))

fig.update_traces( mode='lines+markers')
fig.update_layout(title_text = 'Movies/TV Show release yearly Trend' )
fig.show()

In [30]:
df_4 = df.query("release_year >= 2007")

df_4 = df_4[["type","month",'release_year', "show_id"]]
df_4 = df_4.groupby(['release_year', 'month', 'type'])['show_id'].count().reset_index()
df_4 = df_4.rename(columns = {"show_id": "total_shows"})
df_4 = df_4.groupby(['month', 'type'])['total_shows'].mean().reset_index()



fig  = px.line(df_4, x='month', y='total_shows', color = 'type',
      title='All years Movies/TV Show release Month Trend')
fig.show()

In [31]:
df_4 = df.query("release_year >= 2007")

df_4 = df_4[["type","month",'release_year', "show_id"]]
df_4 = df_4.groupby(['release_year', 'month', 'type'])['show_id'].count().reset_index()
df_4 = df_4.rename(columns = {"show_id": "total_shows"})
df_4 = df_4.groupby(['month', 'type'])['total_shows'].mean().reset_index()

df_4_movie = df_4.query("type == 'Movie'")
df_4_show = df_4.query("type == 'TV Show'")

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=  df_4_movie['month'],
    y= df_4_movie['total_shows'],
    showlegend=True,
    text = df_4_movie['total_shows'], 
    name='Movie',
    marker_color='Maroon'    
    
))
fig.add_trace(go.Scatter(
    x=  df_4_show['month'],
    y= df_4_show['total_shows'],
    showlegend=True,
    text = df_4_show['total_shows'], 
    name='TV Show',
    marker_color='Grey' 
))

fig.update_traces( mode='lines+markers')
fig.update_layout(title_text = 'Movies/TV Shows average release monthly trend' )
fig.show()



In [32]:
def trend_yearwise(year):
    
    title = (f'Movies/TV Show release Month Trend for year {year}' )
    df_6 = df.query("release_year == @year")
    df_6 = df_6.groupby(["type","month"])["show_id"].count().reset_index()
    df_6_movie = df_6.query("type == 'Movie'")
    df_6_show = df_6.query("type == 'TV Show'")
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
    x=  df_6_movie['month'],
    y= df_6_movie['show_id'],
    showlegend=True,
    text = df_6_movie['show_id'], 
    name='Movie',
    marker_color='Maroon'    

    ))
    fig.add_trace(go.Scatter(
    x=  df_6_show['month'],
    y= df_6_show['show_id'],
    showlegend=True,
    text = df_6_show['show_id'], 
    name='TV Show',
    marker_color='Grey' 
    ))

    fig.update_traces( mode='lines+markers')
    fig.update_layout(title_text =  title )
    fig.show()
    
trend_yearwise(2019)    

In [ ]:
df_genre_viz = df_genre[["title", "type", "listed_in"]]
df_genre_viz = df_genre_viz.groupby(['listed_in', 'type'])["title"].count().reset_index().sort_values('title')
df_genre_viz = df_genre_viz.rename(columns = {"title": "movies_count", "listed_in": "Genre"})

df_genre_movie = df_genre_viz.query("type == 'Movie'")
df_genre_tvshow = df_genre_viz.query("type == 'TV Show'")

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'bar'}]],
                   subplot_titles = ['For which Genre the maximum Movies are uploaded?', 'For which Genre the maximum Shows are uploaded? ' ],
                   horizontal_spacing = 0.3)
fig.add_trace(
    
    go.Bar(x= df_genre_movie['movies_count'], y= df_genre_movie['Genre'], orientation = 'h', marker_color='Maroon',  name="Movie",
           text = df_type['count_showids'], textposition='auto'),
    row=1, col=1)

fig.add_trace(
    
    go.Bar(x= df_genre_tvshow['movies_count'], y= df_genre_tvshow['Genre'], orientation = 'h', marker_color = 'Grey', name = "TV Show"),
    row=1, col=2)

fig.update_layout( height = 600)
fig.show()

In [34]:
df_9 = df.query("type == 'TV Show'")
df_9 = df_9[[ "title", "duration"]]
df_9 = df_9.groupby(['duration'])["title"].count().reset_index().sort_values('title', ascending = False)
#df_9 = df_9['duration'].replace("seasons", "")
df_9 = df_9.rename(columns = {"title": "TV Shows", "duration" : "Seasons"})


df_10 = df.query("type == 'Movie'")
df_10['duration'] = df_10['duration'].fillna("0")
df_10['duration'] = df_10['duration'].str.split(" ").str[0].astype(int)




fig_show = px.bar(df_9, x='Seasons', y='TV Shows', color_discrete_sequence=['grey'],
       title='TV Shows seasons ')
fig_Movie = px.histogram(df_10, x="duration" , nbins = 20, color_discrete_sequence=px.colors.sequential.RdBu
                  , title = "Movie Duration")

fig_Movie.show()
fig_show.show()


C:\Users\geeta\AppData\Local\Temp\ipykernel_11564\205072209.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\geeta\AppData\Local\Temp\ipykernel_11564\205072209.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

